## 4.2 Saving models
In this notebook we save a model (random Forest) for each windfarm for later use with the ```dump()```
function of Pickle. Every windfarm has an individual model with the hyperparameters und feature combintaion best
for each windfarm.

In [2]:
# load modules
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from modeling.functions import train_test_split_features, scaler_func
from modeling.features import get_feature_combinations
import pickle
import ast
from sklearn.preprocessing import MinMaxScaler

In [4]:
# loading the dataset
data = pd.read_csv('../data/GEFCom2014Data/Wind/clean_data.csv')                   

In [5]:
# train-test-split 
data_train = data[data['TIMESTAMP']<='2013-07-01 00:00:00']
data_test = data[data['TIMESTAMP']>'2013-07-01 00:00:00']

In [6]:
# load the model we use to make the predictions
model_params = pd.DataFrame()
model_params = pd.read_csv(f'../results/RandomForestRegressor.csv')

In [7]:
for zone in data.ZONEID.unique():
    # feature combination
    fc = model_params[model_params.ZONE == 'ZONE{}'.format(str(zone))]['FC'].values[0]

    # train test split and scale
    X_train, X_test, y_train, y_test = train_test_split_features(data_train, data_test, zone, get_feature_combinations()[fc])  
    X_train, X_test = scaler_func(X_train, X_train, MinMaxScaler()) 

    # hyperparameters
    best_params = model_params[model_params.ZONE == 'ZONE{}'.format(str(zone))]['BEST_PARAMS'].values[0]
  
    
    # define and fit model
    model = RandomForestRegressor()
    model.set_params(**ast.literal_eval(best_params))        
    model.fit(X_train, y_train)

    # save the model
    pickle.dump(model, open('../models/model{}.pkl'.format(zone), 'wb'))

We have saved the models as .pkl. <br>
We have logged the best hyperparamters and feature combinations to mlflow.
So the models can be used by either loading the .pkl model, or loading the model from mlfflow to make predictions without training again. 